In [2]:
from argparse import Namespace
import torch

In [3]:

# <------------- Hyperparameters/Config ------------->
config = Namespace(
    DEVICE = torch.device("mps"), # apple silicon M series
    NUM_WORKERS = 2,
    PIN_MEMORY = True,
    
    EPOCHS = 1,
    LEARNING_RATE = 2e-5,
    BATCH_SIZE = 64,
    WEIGHT_DECAY = 0,


    LOAD_MODEL = False,
    LOAD_MODEL_FILE = None,
    
    DATASET_DIR = "./data", # root path to the dataset dir
    IMAGE_SIZE = 448,

    C = 3, # how many classes in the dataset
    B = 2, # how many bounding boxes does the model perdict per cell
    S = 7, # split_size, how to split the image, 7x7=49 grid cells,
    IOU_THRESHOLD = 0.5, # the iou threshold when comparing bounding boxes for NMS
    MIN_THRESHOLD = 0.4 # the minimal confidence to keep a predicted bounding box
)

config.NUM_NODES_PER_CELL = config.C + 5 * config.B # The total number of nodes per cell, which would be the size ==> [with_mask, without_mask, mask_worn_incorrectly, pc_1, bbox1_x_y_w_h, pc_2, bbox2_x_y_w_h] = 13 nodes.
config.NUM_NODES_PER_IMG = config.S * config.S * (config.C + config.B * 5) # number of nodes that each image has. If S=7 C=3 B= 2 ==+> 7*7 * (3 + 2 * 5) = 637, also 13 * 49 = 637

In [4]:
config

Namespace(DEVICE=device(type='mps'), NUM_WORKERS=2, PIN_MEMORY=True, EPOCHS=1, LEARNING_RATE=2e-05, BATCH_SIZE=64, WEIGHT_DECAY=0, LOAD_MODEL=False, LOAD_MODEL_FILE=None, DATASET_DIR='./data', IMAGE_SIZE=448, C=3, B=2, S=7, IOU_THRESHOLD=0.5, MIN_THRESHOLD=0.4, NUM_NODES_PER_CELL=13, NUM_NODES_PER_IMG=637)

In [95]:
batch_size = 1  # or any number you want
tensor = torch.randn(batch_size, 7, 7, 13)
tensor.shape 

torch.Size([1, 7, 7, 13])

In [96]:
tensor[0][0][0]

tensor([-1.3619, -0.1905, -0.3031, -0.2435, -0.0026,  0.8839, -0.2493, -0.6624,
         0.8513, -0.6011, -1.9173,  0.0658,  0.2680])

In [98]:
tensor[0][0][0][..., :3]

tensor([-1.3619, -0.1905, -0.3031])

In [108]:
from datetime import datetime



def generate_model_file_name(model_name, dataset_name, task, epochnum):
    # <model_name>_<dataset>_<task>_<details>_<epoch|date|version>.pt
    # example: resnet50_cifar10_classification_lr0.001_epoch20.pt
    
    date = datetime.now().strftime("%Y-%m-%d-%Hh_%Mm")
    return f"{model_name}_{dataset_name}_{task}_epoch{epochnum}_{date}.pt"
    

In [110]:
generate_model_file_name("Yolov1", "facemask", "objectDetection", 12 )

'Yolov1_facemask_objectDetection_epoch12_2025-04-08-21h_51m.pt'